# Распознавание рукописных цифр
В данной работе напишу нейронку на базе Pytorch CNN для распознавания собственных рукописных цифр. Сложность в том что, обученная сеть на MNIST на моих цифрах показывает низкие результаты. 

In [1]:
import os
print("Current working directory:", os.getcwd()) # current way
# directory = os.path.join(os.path.dirname(__file__), "my_folder")

Current working directory: c:\Python3\PyIntel\img_transform\Digits


In [2]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder
# from torchvision import models

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch, "xpu") and torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

# xpu
print("Using device:", device)

Using device: xpu


## Model
здесь в результате тестов подобрана максимально эффективная модель для моих рукописных цифр

In [14]:
conv_model2 = nn.Sequential(
    nn.Conv2d(3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),

    nn.Conv2d(16, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),

    nn.Conv2d(16, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),

    nn.Conv2d(32, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2), # batch, out, 7, 7

    nn.Flatten(1), # batch, out
    nn.Linear(32*7*7, 32, bias=False),
    nn.BatchNorm1d(32),
    nn.ReLU(inplace=True),

    nn.Linear(32, 32, bias=False),
    nn.BatchNorm1d(32),
    nn.ReLU(inplace=True),
    nn.Linear(32, 10)
)

In [15]:
model = conv_model2
model.to(device)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Flatten(start_dim=1, 

### формирование обучающих данных

In [16]:
transforms = tfs.Compose([tfs.ToImage(),
                          tfs.ToDtype(torch.float32, scale=True),
                          ])

d_train = ImageFolder("dataset/train", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

x_, y_ = next(iter(train_data))
x_[0].shape

torch.Size([3, 28, 28])

### train 
обучение и сохранение весов

In [17]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()
epochs = 30
model.train()

for _e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean
        train_tqdm.set_description(f"Epoch [{_e+1}/{epochs}], loss_mean={loss_mean:.3f}")

Epoch [30/30], loss_mean=0.005: 100%|██████████| 1875/1875 [00:40<00:00, 46.51it/s]


## Restore!

In [18]:
# save
# torch.save(model.state_dict(), f'model_1_weight_{int(Q*10000)}.pth')

In [19]:
# model.load_state_dict(torch.load('model_weight_9930.pth', map_location='xpu'))

### test
результаты теста на обучающей выбрке

In [20]:
d_test = ImageFolder("dataset/test", transform=transforms)
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False)

Q = 0

# тестирование обученной НС
model.eval()

for x_test, y_test in test_data:
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1)
        Q += torch.sum(p == y_test).item()

Q /= len(d_test)
print(Q)

0.9946


In [21]:
transform = tfs.Compose([
    tfs.ToImage(),
    tfs.Resize((28, 28)),                  # подгоняем под MNIST
    tfs.RandomInvert(p=1.0),               # инвертируем цвет
    tfs.ToDtype(torch.float32, scale=True)
])
# transform(img).shape

In [22]:
# img = Image.open("images/im_2.png")
# img = img.convert("RGB")
# img = img.resize((28, 28))
# # tr = tfs.Compose([tfs.RandomInvert(p=1.0), tfs.Grayscale()])
# tr = tfs.Compose([tfs.RandomInvert(p=1.0)])
# img = tr(img)
# img = transform(img)
# img = img.to(device)
# torch.argmax(model(img.unsqueeze(0)).squeeze())

### test
тест на моих цифрах

In [23]:
res_test = {}
with torch.no_grad():
    for i in range(10):
        img = Image.open(f"images/im_{i}.png")
        img = img.convert("RGB")
        tensor_img = transform(img)
        tensor_img = tensor_img.to(device)
        # res = model(tensor_img)
        res = model(tensor_img.unsqueeze(0)).squeeze()
        # res = res.softmax(dim=0) # .sort(descending=True)
        res = torch.argmax(res)
        res_test[i] = res.item()
        # res_test[i] = res
        
    
res_test

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

просмотр уверенности модели в предсказаниии

In [24]:
# просмотр уверенности модели в предсказаниии

res_test = {}
with torch.no_grad():
    for i in range(10):
        img = Image.open(f"images/im_{i}.png")
        img = img.convert("RGB")
        tensor_img = transform(img)
        tensor_img = tensor_img.to(device)
        # res = model(tensor_img)
        res = model(tensor_img.unsqueeze(0)).squeeze()
        print(i, res)
        # res = res.softmax(dim=0) # .sort(descending=True)
        res = torch.argmax(res)
        res_test[i] = res.item()
        # res_test[i] = res

0 tensor([  5.1438,  -5.5931,  -4.6117, -14.3769,  -6.4987,  -8.8049,  -3.0568,
         -2.0799,  -8.2567,  -7.7662], device='xpu:0')
1 tensor([-18.0151,   6.8636, -13.7433, -18.1379, -15.1786, -13.8746, -13.2797,
          0.1517,  -9.6195, -14.0596], device='xpu:0')
2 tensor([-12.8619,  -8.9557,  12.7133, -13.1203,  -9.6874, -18.5373, -13.9622,
         -6.6759,  -5.9096,  -9.3503], device='xpu:0')
3 tensor([-12.5446,  -6.9928,  -9.6180,   9.1124, -13.5135,  -6.9957, -14.4150,
         -2.0937, -19.3546, -16.2413], device='xpu:0')
4 tensor([ -8.7306,  -9.5899, -13.1923, -16.4296,  10.5303, -10.5634,  -6.9361,
         -8.6767, -14.9485,  -5.9290], device='xpu:0')
5 tensor([-14.7821, -10.3882, -17.2775, -11.1291, -14.9441,  13.0621,  -6.5978,
        -10.1905, -14.7512, -11.9467], device='xpu:0')
6 tensor([ -4.6542,  -6.5027, -12.8920, -16.1850,  -7.9885,   1.8468,   4.4932,
         -5.7202,  -7.7378,  -5.8056], device='xpu:0')
7 tensor([-25.2367,  -3.2251,  -7.8446, -14.1211, -12.7

В результате ряда экспериментов подобрана модель и приведен пример распознавания цифр не из обучающей выборки